In [1]:
import cv2
import numpy as np # We'll be storing our data as numpy arrays
import os # For handling directories
from PIL import Image # For handling the images
import matplotlib.pyplot as plt
import matplotlib.image as mpimg # Plotting

In [2]:
%matplotlib inline

In [3]:
pwd

'D:\\gesture-recognition'

In [4]:
lookup = dict()
reverselookup = dict()
count = 0
for j in os.listdir('data/'):
    if not j.startswith('.'): # If running this code locally, this is to 
                              # ensure you aren't reading in hidden folders
        lookup[j] = count
        reverselookup[count] = j
        count = count + 1


In [5]:
lookup

{'fist': 0, 'index': 1, 'L': 2, 'ok': 3, 'palm': 4, 'peace': 5}

In [6]:
reverselookup

{0: 'fist', 1: 'index', 2: 'L', 3: 'ok', 4: 'palm', 5: 'peace'}

In [7]:
x_data = []
y_data = []
datacount = 0 
 # Loop over the ten top-level folders
for j in os.listdir('data/'):
    if not j.startswith('.'): # Again avoid hidden folders
        count = 0 # To tally images of a given gesture
        for k in os.listdir('data/' + j + '/'):
                            # Loop over the images
            img = Image.open('data/' + j + '/' + k)
                            # Read in and convert to greyscale
            arr = np.array(img)
            x_data.append(arr) 
            count = count + 1
        y_values = np.full((count, 1), lookup[j]) 
        y_data.append(y_values)
        datacount = datacount + count

In [8]:
x_data = np.array(x_data, dtype = 'float32')
y_data = np.array(y_data)
y_data = y_data.reshape(datacount, 1)

In [9]:
x_data.shape

(900, 300, 300)

In [10]:
y_data.shape

(900, 1)

In [14]:
from keras.utils import to_categorical
y_data = to_categorical(y_data)

Using TensorFlow backend.
d:\mltensor\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
d:\mltensor\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
d:\mltensor\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
d:\mltensor\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as

In [15]:
y_data.shape

(900, 6)

In [16]:
x_data = x_data.reshape((datacount, 300, 300, 1))
x_data /= 255
x_data.shape

(900, 300, 300, 1)

In [17]:
from sklearn.model_selection import train_test_split
x_train,x_further,y_train,y_further = train_test_split(x_data,y_data,test_size = 0.2)
x_validate,x_test,y_validate,y_test = train_test_split(x_further,y_further,test_size = 0.5)

In [18]:
from keras import layers
from keras import models

In [19]:
model=models.Sequential()
model.add(layers.Conv2D(32, (5, 5), strides=(2, 2), activation='relu', input_shape=(300, 300,1))) 
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu')) 
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(6, activation='softmax'))

W1009 10:29:15.111840  8304 deprecation_wrapper.py:119] From d:\mltensor\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1009 10:29:15.137838  8304 deprecation_wrapper.py:119] From d:\mltensor\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1009 10:29:15.142841  8304 deprecation_wrapper.py:119] From d:\mltensor\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1009 10:29:15.184841  8304 deprecation_wrapper.py:119] From d:\mltensor\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



In [20]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      832       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 64)        36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 18496)             0         
__________

In [144]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x_train, y_train, epochs=50, batch_size=64, verbose=1, validation_data=(x_validate, y_validate))

Train on 720 samples, validate on 90 samples
Epoch 1/50
720/720 [==============================] - 2s 2ms/step - loss: 0.9113 - acc: 0.9222 - val_loss: 0.0641 - val_acc: 0.9667
Epoch 2/50
720/720 [==============================] - 1s 2ms/step - loss: 0.0014 - acc: 1.0000 - val_loss: 0.0891 - val_acc: 0.9667
Epoch 3/50
720/720 [==============================] - 1s 2ms/step - loss: 4.2491e-04 - acc: 1.0000 - val_loss: 0.0881 - val_acc: 0.9778
Epoch 4/50
720/720 [==============================] - 1s 2ms/step - loss: 2.0092e-04 - acc: 1.0000 - val_loss: 0.1087 - val_acc: 0.9667
Epoch 5/50
720/720 [==============================] - 1s 2ms/step - loss: 1.0315e-04 - acc: 1.0000 - val_loss: 0.0958 - val_acc: 0.9667
Epoch 6/50
720/720 [==============================] - 1s 2ms/step - loss: 4.9769e-05 - acc: 1.0000 - val_loss: 0.0886 - val_acc: 0.9778
Epoch 7/50
720/720 [==============================] - 1s 2ms/step - loss: 2.5497e-05 - acc: 1.0000 - val_loss: 0.0858 - val_acc: 0.9778
Epoch 8/50


In [145]:
[loss, acc] = model.evaluate(x_validate,y_validate,verbose=1)
print("Accuracy:" + str(acc))

90/90 [==============================] - 0s 978us/step
Accuracy:0.9777777777777777


In [256]:
new = Image.open('test/110.png')
a = np.array(new)
a = a/255
a = a.reshape(1, 300, 300, 1)

In [257]:
result = model.predict(a)

In [258]:
result

array([[9.9900275e-01, 9.8406861e-04, 2.5095383e-06, 2.5871447e-09,
        1.0640510e-05, 1.3004777e-07]], dtype=float32)

In [259]:
lookup

{'fist': 0, 'index': 1, 'L': 2, 'ok': 3, 'palm': 4, 'peace': 5}

In [260]:
label = {}

In [261]:
for y, x in lookup.items():
    label[x] = y

In [262]:
label

{0: 'fist', 1: 'index', 2: 'L', 3: 'ok', 4: 'palm', 5: 'peace'}

In [263]:
l = np.where(result[0] == np.amax(result[0]))

In [264]:
print(label[l[0][0]])

fist


In [265]:
print(result[0][l][0])

0.99900275


In [266]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
